# LLM-based Recommender System - Complete Notebook

This notebook contains all the code needed to train and predict with the LLM-based recommender system.

## Overview
- **Stage A**: Pretrains user/item embeddings using collaborative filtering losses with a full-finetuned LLM backbone
- Uses embeddings as soft tokens injected into the LLM
- Efficient dot-product scoring for next-item prediction


In [1]:
# ============================================================================
# IMPORTS AND SETUP
# ============================================================================

import os
import sys
import json
import pickle
import gzip
import random
import shutil
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
from collections import defaultdict

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast, GradScaler
from torch.optim import AdamW
from tqdm import tqdm
import yaml
from transformers import AutoModel, AutoConfig, AutoTokenizer

# Set device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


/home/jsawant/.conda/envs/jay2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0


## Configuration

Set your configuration here. You can modify these values to customize training and prediction.


In [2]:

# ============================================================================
# CONFIGURATION
# ============================================================================

# Data paths
USER_SEQUENCES_PATH = "/space/mcdonald-syn01/1/projects/jsawant/llm_recommender_2/dataset/filtered_user_sequences.jsonl.gz"
ITEM_METADATA_PATH = "/space/mcdonald-syn01/1/projects/jsawant/llm_recommender_2/dataset/filtered_Kindle_Store.jsonl.gz"
DATA_OUTPUT_DIR = "./data/processed_notebook"

# Model configuration
CONFIG = {
  "data": {
        'user_sequences_path': USER_SEQUENCES_PATH,
        'item_metadata_path': ITEM_METADATA_PATH,
    "train_ratio": 0.8,
    "val_ratio": 0.1,
    "min_sequence_length": 3,
    "max_sequence_length": 50,
    "seed": 42,
    "max_train_sequences": 100000
  },
  "model": {
    "base_llm": "gpt2",
    "embedding_dim": 128,
    "freeze_llm_stage_a": False,
    "freeze_llm_stage_b": False,
    "random_init_stage_a_llm": False,
    "use_lora": False,
    "lora_r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "use_embedding_fusion": False,
    "fusion_weight": 0.7
  },
  "embeddings": {
    "lambda_c": 0.001,
    "lambda_t": 0.001,
    "temperature": 0.07
  },
  "stage_a": {
    "epochs": 10,
    "batch_size": 64,
    "learning_rate": 0.0005,
    "weight_decay": 0.001,
    "max_grad_norm": 1.0,
    "loss_weights": {
      "collaborative": 1.0,
      "content": 0.0,
      "contrastive": 0.0,
      "cf_bpr": 0.2,
      "regularization": 0.0
    },
    "collaborative": {
      "negative_samples": 30,
      "use_bpr_loss": True
    },
    "content": {
      "max_text_length": 128
    }
  },
  "training": {
    "device": str(device),
    "mixed_precision": "no",
    "logging_steps": 500,
    "eval_steps": 10000,
    "save_steps": 10000
  }
}

# Output directories
CHECKPOINT_DIR = "./checkpoints/notebook_training"
Path(CHECKPOINT_DIR).mkdir(parents=True, exist_ok=True)
Path(DATA_OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print("Configuration loaded!")


Configuration loaded!


## Utility Functions and Classes

All the core classes and functions from the codebase.


In [3]:
# ============================================================================
# DATA PREPROCESSING
# ============================================================================

class DataPreprocessor:
    """Preprocesses user sequences and item metadata for LLM-based recommendation."""
    
    def __init__(
        self,
        user_sequences_path: str,
        item_metadata_path: Optional[str] = None,
        min_sequence_length: int = 5,
        max_sequence_length: int = 50,
        train_ratio: float = 0.8,
        val_ratio: float = 0.1,
        seed: int = 42,
        max_train_sequences: Optional[int] = None
    ):
        self.user_sequences_path = user_sequences_path
        self.item_metadata_path = item_metadata_path
        self.min_sequence_length = min_sequence_length
        self.max_sequence_length = max_sequence_length
        self.train_ratio = train_ratio
        self.val_ratio = val_ratio
        self.test_ratio = 1.0 - train_ratio - val_ratio
        self.seed = seed
        self.max_train_sequences = max_train_sequences
        
        np.random.seed(seed)
        
        # Mappings
        self.user2id: Dict[str, int] = {}
        self.item2id: Dict[str, int] = {}
        self.id2user: Dict[int, str] = {}
        self.id2item: Dict[int, str] = {}
        
        # Data
        self.user_sequences: Dict[int, List[Dict]] = {}
        self.item_metadata: Dict[int, Dict] = {}
        
    def load_and_preprocess(self) -> Tuple[Dict, Dict, Dict]:
        """Load and preprocess all data."""
        print("Loading user sequences...")
        self._load_user_sequences()
        
        if self.item_metadata_path:
            print("Loading item metadata...")
            self._load_item_metadata()
        
        # Truncate user sequences first if requested
        if self.max_train_sequences is not None:
            print(f"\nTruncating to {self.max_train_sequences} sequences...")
            self._truncate_user_sequences(self.max_train_sequences)
        
        print("Creating train/val/test splits...")
        train_data, val_data, test_data = self._create_splits()
        
        # Remap item IDs to be compact (1-indexed, 0 for padding)
        if self.max_train_sequences is not None:
            print("\nRemapping item IDs to compact range...")
            train_data, val_data, test_data = self._remap_item_ids(
                train_data, val_data, test_data
            )
        
        stats = self._compute_statistics()
        print(f"\nDataset Statistics:")
        print(f"  Users: {stats['num_users']}")
        print(f"  Items: {stats['num_items']}")
        print(f"  Interactions: {stats['num_interactions']}")
        print(f"  Avg sequence length: {stats['avg_seq_length']:.2f}")
        print(f"  Train sequences: {len(train_data)}")
        print(f"  Val sequences: {len(val_data)}")
        print(f"  Test sequences: {len(test_data)}")
        
        return train_data, val_data, test_data
    
    def _load_user_sequences(self):
        """Load user interaction sequences from JSONL file."""
        user_idx = 0
        item_idx = 1  # Start from 1, reserve 0 for padding
        
        with gzip.open(self.user_sequences_path, 'rt', encoding='utf-8') as f:
            for line in tqdm(f, desc="Loading sequences"):
                data = json.loads(line.strip())
                user_id_str = data['user_id']
                sequence = data['sequence']
                
                # Filter by sequence length
                if len(sequence) < self.min_sequence_length:
                    continue
                
                # Map user ID
                if user_id_str not in self.user2id:
                    self.user2id[user_id_str] = user_idx
                    self.id2user[user_idx] = user_id_str
                    user_idx += 1
                
                user_id = self.user2id[user_id_str]
                
                # Map item IDs and process sequence
                processed_sequence = []
                for interaction in sequence:
                    item_id_str = interaction['asin']
                    
                    if item_id_str not in self.item2id:
                        self.item2id[item_id_str] = item_idx
                        self.id2item[item_idx] = item_id_str
                        item_idx += 1
                    
                    item_id = self.item2id[item_id_str]
                    
                    processed_sequence.append({
                        'item_id': item_id,
                        'timestamp': interaction['ts'],
                        'rating': interaction.get('rating', 0.0)
                    })
                
                # Sort by timestamp
                processed_sequence = sorted(processed_sequence, key=lambda x: x['timestamp'])
                
                # Truncate if too long
                if len(processed_sequence) > self.max_sequence_length:
                    processed_sequence = processed_sequence[-self.max_sequence_length:]
                
                self.user_sequences[user_id] = processed_sequence
    
    def _load_item_metadata(self):
        """Load item metadata (titles, descriptions, etc.)."""
        with gzip.open(self.item_metadata_path, 'rt', encoding='utf-8') as f:
            for line in tqdm(f, desc="Loading metadata"):
                data = json.loads(line.strip())
                item_id_str = data.get('parent_asin') or data.get('asin')
                
                if item_id_str in self.item2id:
                    item_id = self.item2id[item_id_str]
                    
                    # Extract relevant metadata
                    self.item_metadata[item_id] = {
                        'title': data.get('title', ''),
                        'description': ' '.join(data.get('description', [])) if isinstance(data.get('description'), list) else data.get('description', ''),
                        'categories': data.get('categories', []),
                        'price': data.get('price', 0.0),
                        'average_rating': data.get('average_rating', 0.0)
                    }
    
    def _create_splits(self) -> Tuple[Dict, Dict, Dict]:
        """Create train/validation/test splits using leave-one-out strategy."""
        train_data = {}
        val_data = {}
        test_data = {}
        
        for user_id, sequence in self.user_sequences.items():
            seq_len = len(sequence)
            
            if seq_len < 3:
                continue
            
            # Leave-one-out split: last item for test, second-to-last for val
            train_seq = sequence[:-2]  # All items up to T-2
            val_item = sequence[-2]     # Item at T-1
            test_item = sequence[-1]    # Item at T
            
            if len(train_seq) >= self.min_sequence_length - 1:
                # Training: Full sequence for autoregressive training
                train_data[user_id] = {
                    'sequence': train_seq,
                    'is_autoregressive': True
                }
                
                # Validation: Predict single next item
                val_data[user_id] = {
                    'sequence': train_seq,
                    'target': val_item,
                    'is_autoregressive': False
                }
                
                # Test: Predict single next item
                test_data[user_id] = {
                    'sequence': sequence[:-1],  # Include validation item
                    'target': test_item,
                    'is_autoregressive': False
                }
        
        return train_data, val_data, test_data
    
    def _truncate_user_sequences(self, max_sequences: int):
        """Truncate user_sequences to max_sequences before creating splits."""
        user_ids = list(self.user_sequences.keys())
        if len(user_ids) > max_sequences:
            np.random.shuffle(user_ids)
            user_ids = user_ids[:max_sequences]
            self.user_sequences = {uid: self.user_sequences[uid] for uid in user_ids}
            print(f"  Truncated to {len(self.user_sequences)} user sequences")
            
            # Update user mappings
            kept_user_ids = set(user_ids)
            new_user2id = {}
            new_id2user = {}
            new_user_idx = 0
            
            for old_user_id in sorted(kept_user_ids):
                old_user_str = self.id2user[old_user_id]
                new_user2id[old_user_str] = new_user_idx
                new_id2user[new_user_idx] = old_user_str
                new_user_idx += 1
            
            # Remap user IDs in sequences
            remapped_sequences = {}
            for old_user_id, sequence in self.user_sequences.items():
                old_user_str = self.id2user[old_user_id]
                new_user_id = new_user2id[old_user_str]
                remapped_sequences[new_user_id] = sequence
            
            self.user_sequences = remapped_sequences
            self.user2id = new_user2id
            self.id2user = new_id2user
    
    def _remap_item_ids(
        self,
        train_data: Dict,
        val_data: Dict,
        test_data: Dict
    ) -> Tuple[Dict, Dict, Dict]:
        """Remap item IDs to compact range (1-indexed, 0 reserved for padding)."""
        # Collect all items that appear in final dataset
        all_items = set()
        
        for user_data in train_data.values():
            for item in user_data['sequence']:
                all_items.add(item['item_id'])
        
        for user_data in val_data.values():
            for item in user_data['sequence']:
                all_items.add(item['item_id'])
            all_items.add(user_data['target']['item_id'])
        
        for user_data in test_data.values():
            for item in user_data['sequence']:
                all_items.add(item['item_id'])
            all_items.add(user_data['target']['item_id'])
        
        print(f"  Found {len(all_items)} unique items in final dataset")
        
        # Create mapping: old_item_id -> new_item_id
        old_to_new_item = {}
        new_item_idx = 1  # Start from 1, reserve 0 for padding
        
        sorted_items = sorted(all_items)
        for old_item_id in sorted_items:
            old_to_new_item[old_item_id] = new_item_idx
            new_item_idx += 1
        
        # Update item2id and id2item mappings
        new_item2id = {}
        new_id2item = {}
        for old_item_id, new_item_id in old_to_new_item.items():
            old_item_str = self.id2item[old_item_id]
            new_item2id[old_item_str] = new_item_id
            new_id2item[new_item_id] = old_item_str
        
        self.item2id = new_item2id
        self.id2item = new_id2item
        
        # Remap item IDs in all data
        def remap_sequence(sequence):
            return [{
                **item,
                'item_id': old_to_new_item[item['item_id']]
            } for item in sequence]
        
        def remap_target(target):
            return {
                **target,
                'item_id': old_to_new_item[target['item_id']]
            }
        
        remapped_train_data = {}
        for user_id, user_data in train_data.items():
            remapped_train_data[user_id] = {
                'sequence': remap_sequence(user_data['sequence']),
                'is_autoregressive': user_data['is_autoregressive']
            }
        
        remapped_val_data = {}
        for user_id, user_data in val_data.items():
            remapped_val_data[user_id] = {
                'sequence': remap_sequence(user_data['sequence']),
                'target': remap_target(user_data['target']),
                'is_autoregressive': user_data['is_autoregressive']
            }
        
        remapped_test_data = {}
        for user_id, user_data in test_data.items():
            remapped_test_data[user_id] = {
                'sequence': remap_sequence(user_data['sequence']),
                'target': remap_target(user_data['target']),
                'is_autoregressive': user_data['is_autoregressive']
            }
        
        # Update item_metadata
        remapped_item_metadata = {}
        for old_item_id, new_item_id in old_to_new_item.items():
            if old_item_id in self.item_metadata:
                remapped_item_metadata[new_item_id] = self.item_metadata[old_item_id]
        
        self.item_metadata = remapped_item_metadata
        
        print(f"  Remapped {len(old_to_new_item)} items to new IDs (1-{len(old_to_new_item)})")
        print(f"  Updated item metadata: {len(self.item_metadata)} items")
        
        return remapped_train_data, remapped_val_data, remapped_test_data
    
    def _compute_statistics(self) -> Dict:
        """Compute dataset statistics."""
        num_users = len(self.user2id)
        num_items = len(self.item2id)
        num_interactions = sum(len(seq) for seq in self.user_sequences.values())
        avg_seq_length = num_interactions / num_users if num_users > 0 else 0
        
        return {
            'num_users': num_users,
            'num_items': num_items,
            'num_interactions': num_interactions,
            'avg_seq_length': avg_seq_length
        }
    
    def save_mappings(self, save_dir: str):
        """Save user/item mappings to disk."""
        save_path = Path(save_dir)
        save_path.mkdir(parents=True, exist_ok=True)
        
        with open(save_path / 'user2id.pkl', 'wb') as f:
            pickle.dump(self.user2id, f)
        
        with open(save_path / 'item2id.pkl', 'wb') as f:
            pickle.dump(self.item2id, f)
        
        with open(save_path / 'id2user.pkl', 'wb') as f:
            pickle.dump(self.id2user, f)
        
        with open(save_path / 'id2item.pkl', 'wb') as f:
            pickle.dump(self.id2item, f)
        
        with open(save_path / 'item_metadata.pkl', 'wb') as f:
            pickle.dump(self.item_metadata, f)
        
        print(f"Mappings saved to {save_path}")

print("DataPreprocessor class defined!")


DataPreprocessor class defined!


In [4]:
# ============================================================================
# DATASET CLASSES
# ============================================================================

class RecDataset(Dataset):
    """Dataset for LLM-based recommendation."""
    
    def __init__(
        self,
        data: Dict[int, Dict],
        item_metadata: Dict[int, Dict],
        num_items: int,
        mode: str = 'train',
        negative_samples: int = 5,
        max_seq_length: int = 50,
        eval_negatives: int = 100,
        eval_seed: int = 42
    ):
        self.data = data
        self.item_metadata = item_metadata
        self.num_items = num_items
        self.mode = mode
        self.negative_samples = negative_samples
        self.max_seq_length = max_seq_length
        self.eval_negatives = eval_negatives
        self.eval_seed = eval_seed
        
        self.user_ids = list(data.keys())
        
        # Precompute negative samples for val/test (SASRec protocol: 100 negatives per user)
        if self.mode in ['val', 'test']:
            self.eval_negative_samples = {}
            np.random.seed(eval_seed)
            
            all_items = np.arange(1, self.num_items, dtype=np.int32)
            item_mask = np.ones(self.num_items, dtype=bool)
            item_mask[0] = False
            
            for user_id in tqdm(self.user_ids, desc="Precomputing negative samples"):
                user_data = self.data[user_id]
                sequence = user_data['sequence']
                target = user_data.get('target', {})
                
                user_items_list = [item['item_id'] for item in sequence]
                if target and 'item_id' in target:
                    user_items_list.append(target['item_id'])
                
                item_mask.fill(True)
                item_mask[0] = False
                
                if user_items_list:
                    user_items_arr = np.array(user_items_list, dtype=np.int32)
                    valid_mask = (user_items_arr > 0) & (user_items_arr < self.num_items)
                    item_mask[user_items_arr[valid_mask]] = False
                
                neg_candidates = all_items[item_mask[1:]]
                
                if len(neg_candidates) >= self.eval_negatives:
                    self.eval_negative_samples[user_id] = np.random.choice(
                        neg_candidates, 
                        size=self.eval_negatives, 
                        replace=False
                    ).tolist()
                else:
                    self.eval_negative_samples[user_id] = (
                        neg_candidates.tolist() + 
                        np.random.choice(
                            neg_candidates,
                            size=self.eval_negatives - len(neg_candidates),
                            replace=True
                        ).tolist()
                    )
        
    def __len__(self) -> int:
        return len(self.user_ids)
    
    def __getitem__(self, idx: int) -> Dict:
        user_id = self.user_ids[idx]
        user_data = self.data[user_id]
        
        sequence = user_data['sequence']
        is_autoregressive = user_data.get('is_autoregressive', False)
        
        item_ids = [item['item_id'] for item in sequence]
        ratings = [item['rating'] for item in sequence]
        
        if len(item_ids) > self.max_seq_length:
            item_ids = item_ids[-self.max_seq_length:]
            ratings = ratings[-self.max_seq_length:]
        
        seq_length = len(item_ids)
        
        if is_autoregressive:
            target_item_id = None
            target_item_ids = item_ids[1:] + [0]
        else:
            target = user_data['target']
            target_item_id = target['item_id']
            target_item_ids = None
        
        negative_items = []
        if self.mode == 'train':
            if is_autoregressive:
                user_items = set(item_ids)
            else:
                user_items = set(item_ids + [target_item_id])
            
            neg_candidates = list(set(range(1, self.num_items)) - user_items)
            
            if len(neg_candidates) >= self.negative_samples:
                negative_items = random.sample(neg_candidates, self.negative_samples)
            else:
                negative_items = neg_candidates + random.choices(
                    neg_candidates, 
                    k=self.negative_samples - len(neg_candidates)
                )
        elif self.mode in ['val', 'test']:
            negative_items = self.eval_negative_samples[user_id]
        
        return {
            'user_id': user_id,
            'item_ids': item_ids,
            'ratings': ratings,
            'seq_length': seq_length,
            'target_item_id': target_item_id,
            'target_item_ids': target_item_ids,
            'negative_items': negative_items,
            'is_autoregressive': is_autoregressive
        }


class RecDataModule:
    """Data module for handling train/val/test data loaders."""
    
    def __init__(
        self,
        train_data: Dict,
        val_data: Dict,
        test_data: Dict,
        item_metadata: Dict,
        num_items: int,
        tokenizer = None,
        batch_size: int = 32,
        num_workers: int = 4,
        negative_samples: int = 5,
        max_seq_length: int = 50,
        max_text_length: int = 128,
        eval_seed: int = 42
    ):
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data
        self.item_metadata = item_metadata
        self.num_items = num_items
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.negative_samples = negative_samples
        self.max_seq_length = max_seq_length
        self.max_text_length = max_text_length
        self.eval_seed = eval_seed
        
        self.train_dataset = RecDataset(
            train_data, item_metadata, num_items, 
            mode='train', 
            negative_samples=negative_samples,
            max_seq_length=max_seq_length,
            eval_seed=eval_seed
        )
        
        self.val_dataset = RecDataset(
            val_data, item_metadata, num_items, 
            mode='val',
            max_seq_length=max_seq_length,
            eval_seed=eval_seed
        )
        
        self.test_dataset = RecDataset(
            test_data, item_metadata, num_items, 
            mode='test',
            max_seq_length=max_seq_length,
            eval_seed=eval_seed
        )
    
    def collate_fn(self, batch: List[Dict]) -> Dict:
        """Collate function for batching."""
        user_ids = [item['user_id'] for item in batch]
        seq_lengths = [item['seq_length'] for item in batch]
        is_autoregressive = batch[0]['is_autoregressive']
        
        max_len = max(seq_lengths)
        
        item_ids_padded = []
        ratings_padded = []
        attention_mask = []
        target_item_ids_padded = []
        
        for item in batch:
            item_ids = item['item_ids']
            ratings = item['ratings']
            pad_len = max_len - len(item_ids)
            
            item_ids_padded.append(item_ids + [0] * pad_len)
            ratings_padded.append(ratings + [0.0] * pad_len)
            attention_mask.append([1] * len(item_ids) + [0] * pad_len)
            
            if is_autoregressive:
                target_ids = item['target_item_ids']
                target_item_ids_padded.append(target_ids + [0] * pad_len)
        
        if is_autoregressive:
            target_item_ids = torch.LongTensor(target_item_ids_padded)
        else:
            target_item_ids = torch.LongTensor([item['target_item_id'] for item in batch])
        
        negative_items = []
        if batch[0]['negative_items']:
            negative_items = [item['negative_items'] for item in batch]
        
        return {
            'user_ids': torch.LongTensor(user_ids),
            'item_ids': torch.LongTensor(item_ids_padded),
            'ratings': torch.FloatTensor(ratings_padded),
            'attention_mask': torch.LongTensor(attention_mask),
            'target_item_ids': target_item_ids,
            'negative_items': torch.LongTensor(negative_items) if negative_items else None,
            'seq_lengths': torch.LongTensor(seq_lengths),
            'is_autoregressive': is_autoregressive
        }
    
    def train_dataloader(self) -> DataLoader:
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            collate_fn=self.collate_fn,
            pin_memory=True
        )
    
    def val_dataloader(self) -> DataLoader:
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            collate_fn=self.collate_fn,
            pin_memory=True
        )
    
    def test_dataloader(self) -> DataLoader:
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            collate_fn=self.collate_fn,
            pin_memory=True
        )

print("Dataset classes defined!")


Dataset classes defined!


In [5]:
# ============================================================================
# MODEL CLASSES - EMBEDDINGS
# ============================================================================

class CollaborativeEmbedding(nn.Module):
    """Collaborative embeddings for users and items with projection to LLM hidden space."""
    
    def __init__(
        self,
        num_users: int,
        num_items: int,
        embedding_dim: int,
        llm_hidden_size: int,
        lambda_c: float = 0.01,
        init_method: str = 'normal'
    ):
        super().__init__()
        
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.llm_hidden_size = llm_hidden_size
        self.lambda_c = lambda_c
        
        self.user_embeddings = nn.Embedding(num_users, embedding_dim)
        self.item_embeddings = nn.Embedding(num_items, embedding_dim, padding_idx=0)
        
        self.user_proj = nn.Linear(embedding_dim, llm_hidden_size, bias=False)
        self.item_proj = nn.Linear(embedding_dim, llm_hidden_size, bias=False)
        
        self._initialize_embeddings(init_method)
    
    def _initialize_embeddings(self, method: str):
        if method == 'normal':
            std = (1.0 / self.lambda_c) ** 0.5
            nn.init.normal_(self.user_embeddings.weight, mean=0, std=std)
            nn.init.normal_(self.item_embeddings.weight, mean=0, std=std)
        elif method == 'xavier':
            nn.init.xavier_uniform_(self.user_embeddings.weight)
            nn.init.xavier_uniform_(self.item_embeddings.weight)
        
        nn.init.xavier_uniform_(self.user_proj.weight)
        nn.init.xavier_uniform_(self.item_proj.weight)
    
    def get_user_embeddings(self, user_ids: torch.Tensor) -> tuple:
        raw_embeds = self.user_embeddings(user_ids)
        proj_embeds = self.user_proj(raw_embeds)
        return raw_embeds, proj_embeds
    
    def get_item_embeddings(self, item_ids: torch.Tensor) -> tuple:
        raw_embeds = self.item_embeddings(item_ids)
        proj_embeds = self.item_proj(raw_embeds)
        return raw_embeds, proj_embeds
    
    def forward(
        self,
        user_ids: Optional[torch.Tensor] = None,
        item_ids: Optional[torch.Tensor] = None,
        project: bool = True
    ) -> tuple:
        user_embeds = None
        item_embeds = None
        
        if user_ids is not None:
            if project:
                _, user_embeds = self.get_user_embeddings(user_ids)
            else:
                user_embeds, _ = self.get_user_embeddings(user_ids)
        
        if item_ids is not None:
            if project:
                _, item_embeds = self.get_item_embeddings(item_ids)
            else:
                item_embeds, _ = self.get_item_embeddings(item_ids)
        
        return user_embeds, item_embeds
    
    def get_all_item_embeddings(self, project: bool = False) -> torch.Tensor:
        if project:
            return self.item_proj(self.item_embeddings.weight)
        return self.item_embeddings.weight
    
    def regularization_loss(self) -> torch.Tensor:
        user_reg = (self.user_embeddings.weight ** 2).mean()
        item_reg = (self.item_embeddings.weight ** 2).mean()
        return self.lambda_c * (user_reg + item_reg) / 2


class ItemScoringHead(nn.Module):
    """Scoring head for next-item prediction."""
    
    def __init__(
        self,
        llm_hidden_size: int,
        item_embedding_module: nn.Module,
        content_embedding_module: Optional[nn.Module] = None,
        fusion_weight: float = 0.5,
        use_bias: bool = False
    ):
        super().__init__()
        
        self.llm_hidden_size = llm_hidden_size
        self.item_embedding_module = item_embedding_module
        self.content_embedding_module = content_embedding_module
        self.fusion_weight = fusion_weight
        
        self.output_proj = nn.Linear(llm_hidden_size, item_embedding_module.embedding_dim, bias=use_bias)
        nn.init.xavier_uniform_(self.output_proj.weight)
    
    def forward(
        self,
        llm_output: torch.Tensor,
        candidate_items: Optional[torch.Tensor] = None,
        use_fusion: bool = True
    ) -> torch.Tensor:
        query = self.output_proj(llm_output)
        
        if candidate_items is not None:
            item_embeds_collab, _ = self.item_embedding_module.get_item_embeddings(candidate_items)
            
            if use_fusion and self.content_embedding_module is not None:
                item_embeds_content, _ = self.content_embedding_module.get_item_embeddings(candidate_items)
                item_embeds = (self.fusion_weight * item_embeds_collab + 
                             (1 - self.fusion_weight) * item_embeds_content)
            else:
                item_embeds = item_embeds_collab
            
            scores = torch.bmm(item_embeds, query.unsqueeze(-1)).squeeze(-1)
        else:
            all_item_embeds_collab = self.item_embedding_module.get_all_item_embeddings(project=False)
            
            if use_fusion and self.content_embedding_module is not None:
                all_item_embeds_content = self.content_embedding_module.get_all_item_embeddings(project=False)
                all_item_embeds = (self.fusion_weight * all_item_embeds_collab + 
                                 (1 - self.fusion_weight) * all_item_embeds_content)
            else:
                all_item_embeds = all_item_embeds_collab
            
            scores = torch.matmul(query, all_item_embeds.t())
        
        return scores

print("Embedding classes defined!")


Embedding classes defined!


In [6]:
# ============================================================================
# MODEL CLASSES - STAGE A MODEL
# ============================================================================

class StageAModel(nn.Module):
    """Stage A: Pretrain user/item token embeddings."""
    
    def __init__(
        self,
        base_llm_name: str,
        num_users: int,
        num_items: int,
        embedding_dim: int = 64,
        lambda_c: float = 0.01,
        freeze_llm: bool = True,
        use_bpr_loss: bool = True,
        random_init_llm: bool = False
    ):
        super().__init__()
        
        if random_init_llm:
            llm_config = AutoConfig.from_pretrained(base_llm_name)
            self.llm = AutoModel.from_config(llm_config)
        else:
            self.llm = AutoModel.from_pretrained(base_llm_name)
        self.llm_hidden_size = self.llm.config.hidden_size
        self.vocab_size = self.llm.config.vocab_size
        
        if freeze_llm:
            for param in self.llm.parameters():
                param.requires_grad = False
        
        self.collab_embeddings = CollaborativeEmbedding(
            num_users=num_users,
            num_items=num_items,
            embedding_dim=embedding_dim,
            llm_hidden_size=self.llm_hidden_size,
            lambda_c=lambda_c,
            init_method='xavier'
        )
        
        self.collab_scoring_head = ItemScoringHead(
            self.llm_hidden_size,
            self.collab_embeddings,
            content_embedding_module=None,
            fusion_weight=1.0
        )
        
        self.use_bpr_loss = use_bpr_loss
    
    def forward_collaborative(
        self,
        user_ids: torch.Tensor,
        item_ids: torch.Tensor,
        attention_mask: torch.Tensor
    ) -> torch.Tensor:
        batch_size = user_ids.size(0)
        
        user_embeds, item_embeds = self.collab_embeddings(
            user_ids=user_ids,
            item_ids=item_ids,
            project=True
        )
        
        user_embeds = user_embeds.unsqueeze(1)
        inputs_embeds = torch.cat([user_embeds, item_embeds], dim=1)
        
        user_mask = torch.ones(batch_size, 1, device=attention_mask.device, dtype=attention_mask.dtype)
        full_attention_mask = torch.cat([user_mask, attention_mask], dim=1)
        
        outputs = self.llm(
            inputs_embeds=inputs_embeds,
            attention_mask=full_attention_mask,
            output_hidden_states=True
        )
        
        last_hidden = outputs.last_hidden_state
        seq_lengths = full_attention_mask.sum(dim=1) - 1
        batch_indices = torch.arange(batch_size, device=last_hidden.device)
        output_hidden = last_hidden[batch_indices, seq_lengths]
        
        return output_hidden
    
    def compute_collaborative_loss(
        self,
        user_ids: torch.Tensor,
        item_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        target_items: torch.Tensor,
        is_autoregressive: bool = False,
        negative_items: Optional[torch.Tensor] = None
    ) -> Dict[str, torch.Tensor]:
        batch_size = user_ids.size(0)
        
        user_embeds, item_embeds = self.collab_embeddings(
            user_ids=user_ids,
            item_ids=item_ids,
            project=True
        )
        
        user_embeds = user_embeds.unsqueeze(1)
        inputs_embeds = torch.cat([user_embeds, item_embeds], dim=1)
        
        user_mask = torch.ones(batch_size, 1, device=attention_mask.device, dtype=attention_mask.dtype)
        full_attention_mask = torch.cat([user_mask, attention_mask], dim=1)
        
        outputs = self.llm(
            inputs_embeds=inputs_embeds,
            attention_mask=full_attention_mask,
            output_hidden_states=True
        )
        
        last_hidden = outputs.last_hidden_state
        
        if is_autoregressive:
            item_hidden = last_hidden[:, 1:, :]
            h_query = self.collab_scoring_head.output_proj(item_hidden)
            all_item_embeds = self.collab_embeddings.get_all_item_embeddings(project=False)
            scores = torch.matmul(h_query, all_item_embeds.t())
            
            scores_flat = scores.reshape(-1, scores.size(-1))
            targets_flat = target_items.reshape(-1)
            
            valid_mask = (attention_mask > 0) & (target_items > 0)
            valid_mask_flat = valid_mask.reshape(-1)
            num_valid = valid_mask_flat.sum()
            
            ce_loss = F.cross_entropy(
                scores_flat,
                targets_flat,
                ignore_index=0,
                reduction='sum'
            )
            ce_loss = ce_loss / num_valid.clamp(min=1)
            
            losses = {'collaborative_ce': ce_loss}
            
            if self.use_bpr_loss and negative_items is not None:
                h_context = self.collab_scoring_head.output_proj(item_hidden)
                _, target_embeds_raw = self.collab_embeddings(item_ids=target_items, project=False)
                _, neg_embeds_raw = self.collab_embeddings(item_ids=negative_items, project=False)
                
                pos_scores_context = torch.sum(h_context * target_embeds_raw, dim=-1)
                neg_scores_context = torch.bmm(h_context, neg_embeds_raw.transpose(1, 2))
                
                pos_scores_expanded = pos_scores_context.unsqueeze(-1)
                bpr_logits_context = pos_scores_expanded - neg_scores_context
                bpr_loss_context = -torch.log(torch.sigmoid(bpr_logits_context) + 1e-8).sum(dim=-1)
                bpr_loss_context = (bpr_loss_context * valid_mask.float()).sum() / num_valid.clamp(min=1)
                losses['bpr_context'] = bpr_loss_context
                
                user_embeds_raw, _ = self.collab_embeddings(user_ids=user_ids, project=False)
                user_embeds_expanded = user_embeds_raw.unsqueeze(1).expand(-1, target_embeds_raw.size(1), -1)
                
                pos_scores_user = torch.sum(user_embeds_expanded * target_embeds_raw, dim=-1)
                neg_scores_user = torch.bmm(user_embeds_expanded, neg_embeds_raw.transpose(1, 2))
                
                pos_scores_user_expanded = pos_scores_user.unsqueeze(-1)
                bpr_logits_user = pos_scores_user_expanded - neg_scores_user
                bpr_loss_user = -torch.log(torch.sigmoid(bpr_logits_user) + 1e-8).sum(dim=-1)
                bpr_loss_user = (bpr_loss_user * valid_mask.float()).sum() / num_valid.clamp(min=1)
                losses['bpr_user'] = bpr_loss_user
                losses['bpr'] = bpr_loss_context + bpr_loss_user
        else:
            seq_lengths = full_attention_mask.sum(dim=1) - 1
            batch_indices = torch.arange(batch_size, device=last_hidden.device)
            llm_output = last_hidden[batch_indices, seq_lengths]
            
            scores = self.collab_scoring_head(llm_output)
            ce_loss = F.cross_entropy(scores, target_items)
            losses = {'collaborative_ce': ce_loss}
            
            if self.use_bpr_loss and negative_items is not None:
                h_context = self.collab_scoring_head.output_proj(llm_output)
                _, target_embeds_raw = self.collab_embeddings(item_ids=target_items, project=False)
                _, neg_embeds_raw = self.collab_embeddings(item_ids=negative_items, project=False)
                
                pos_scores_context = torch.sum(h_context * target_embeds_raw, dim=-1)
                neg_scores_context = torch.sum(h_context.unsqueeze(1) * neg_embeds_raw, dim=-1)
                bpr_loss_context = -torch.log(torch.sigmoid(pos_scores_context.unsqueeze(1) - neg_scores_context) + 1e-8).mean()
                losses['bpr_context'] = bpr_loss_context
                
                user_embeds_raw, _ = self.collab_embeddings(user_ids=user_ids, project=False)
                pos_scores_user = torch.sum(user_embeds_raw * target_embeds_raw, dim=-1)
                neg_scores_user = torch.sum(user_embeds_raw.unsqueeze(1) * neg_embeds_raw, dim=-1)
                bpr_loss_user = -torch.log(torch.sigmoid(pos_scores_user.unsqueeze(1) - neg_scores_user) + 1e-8).mean()
                losses['bpr_user'] = bpr_loss_user
                losses['bpr'] = bpr_loss_context + bpr_loss_user
        
        return losses
    
    def compute_regularization_losses(self) -> Dict[str, torch.Tensor]:
        collab_reg = self.collab_embeddings.regularization_loss()
        return {'collab_regularization': collab_reg}

print("StageAModel class defined!")


StageAModel class defined!


In [7]:
# ============================================================================
# METRICS
# ============================================================================

def hr_at_k_sampled(scores: torch.Tensor, targets: torch.Tensor, k: int) -> float:
    """Compute Hit@K for sampled candidate evaluation (SASRec-style)."""
    topk_idx = scores.topk(k, dim=-1).indices
    hits = (topk_idx == targets.unsqueeze(-1)).any(dim=-1).float()
    return hits.mean().item()


def ndcg_at_k_sampled(scores: torch.Tensor, targets: torch.Tensor, k: int) -> float:
    """Compute NDCG@K for sampled candidate evaluation (SASRec-style)."""
    topk_idx = scores.topk(k, dim=-1).indices
    hits = (topk_idx == targets.unsqueeze(-1))
    
    positions = torch.arange(1, k + 1, device=scores.device, dtype=torch.float32)
    discounts = 1.0 / torch.log2(positions + 1.0)
    
    gains = (hits.float() * discounts.unsqueeze(0)).max(dim=-1).values
    idcg = 1.0 / torch.log2(torch.tensor(2.0, device=scores.device))
    ndcg = gains / idcg
    
    return ndcg.mean().item()


def compute_sampled_metrics(
    scores: torch.Tensor,
    targets: torch.Tensor,
    k_list: List[int] = [1, 5, 10]
) -> Dict[str, float]:
    """Compute SASRec-style sampled metrics."""
    scores = scores.to(torch.float32)
    targets = targets.to(torch.long)
    metrics = {}
    
    for k in k_list:
        metrics[f'hit@{k}'] = hr_at_k_sampled(scores, targets, k)
        metrics[f'ndcg@{k}'] = ndcg_at_k_sampled(scores, targets, k)
    
    return metrics


def print_metrics(metrics: Dict[str, float], prefix: str = ""):
    """Print metrics in a formatted way."""
    print(f"\n{prefix}Metrics:")
    print("-" * 50)
    for metric_name, value in sorted(metrics.items()):
        print(f"  {metric_name}: {value:.4f}")
    print("-" * 50)

print("Metrics functions defined!")


Metrics functions defined!


## Step 1: Data Preprocessing

Run this cell to preprocess your data. This will create train/val/test splits and save the processed data.


In [8]:
# ============================================================================
# DATA PREPROCESSING
# ============================================================================

# Create preprocessor
preprocessor = DataPreprocessor(
    user_sequences_path=CONFIG['data']['user_sequences_path'],
    item_metadata_path=CONFIG['data'].get('item_metadata_path'),
    min_sequence_length=CONFIG['data']['min_sequence_length'],
    max_sequence_length=CONFIG['data']['max_sequence_length'],
    train_ratio=CONFIG['data']['train_ratio'],
    val_ratio=CONFIG['data']['val_ratio'],
    seed=CONFIG['data']['seed'],
    max_train_sequences=CONFIG['data'].get('max_train_sequences')
)

# Load and preprocess data
train_data, val_data, test_data = preprocessor.load_and_preprocess()

# Save processed data
print(f"\nSaving processed data to {DATA_OUTPUT_DIR}...")
with open(Path(DATA_OUTPUT_DIR) / 'train_data.pkl', 'wb') as f:
    pickle.dump(train_data, f)
with open(Path(DATA_OUTPUT_DIR) / 'val_data.pkl', 'wb') as f:
    pickle.dump(val_data, f)
with open(Path(DATA_OUTPUT_DIR) / 'test_data.pkl', 'wb') as f:
    pickle.dump(test_data, f)

# Save mappings
preprocessor.save_mappings(DATA_OUTPUT_DIR)

# Save dataset info
dataset_info = {
    'num_users': len(preprocessor.user2id),
    'num_items': len(preprocessor.item2id) + 1,  # +1 for padding
    'num_train': len(train_data),
    'num_val': len(val_data),
    'num_test': len(test_data)
}

with open(Path(DATA_OUTPUT_DIR) / 'dataset_info.pkl', 'wb') as f:
    pickle.dump(dataset_info, f)

print(f"\nPreprocessing complete! Data saved to {DATA_OUTPUT_DIR}")
print("\nDataset info:")
for key, value in dataset_info.items():
    print(f"  {key}: {value}")

# Store for later use
num_users = dataset_info['num_users']
num_items = dataset_info['num_items']
item_metadata = preprocessor.item_metadata


Loading user sequences...


Loading sequences: 761756it [00:19, 39216.50it/s]


Loading item metadata...


Loading metadata: 20318771it [02:47, 121026.48it/s]



Truncating to 100000 sequences...
  Truncated to 100000 user sequences
Creating train/val/test splits...

Remapping item IDs to compact range...
  Found 389161 unique items in final dataset
  Remapped 389161 items to new IDs (1-389161)
  Updated item metadata: 389161 items

Dataset Statistics:
  Users: 100000
  Items: 389161
  Interactions: 1319558
  Avg sequence length: 13.20
  Train sequences: 100000
  Val sequences: 100000
  Test sequences: 100000

Saving processed data to ./data/processed_notebook...
Mappings saved to data/processed_notebook

Preprocessing complete! Data saved to ./data/processed_notebook

Dataset info:
  num_users: 100000
  num_items: 389162
  num_train: 100000
  num_val: 100000
  num_test: 100000


## Step 1.5: Load Preprocessed Data (Optional)

If you've already preprocessed the data, you can load it directly from disk instead of running preprocessing again.


In [9]:
# ============================================================================
# LOAD PREPROCESSED DATA (OPTIONAL - SKIP PREPROCESSING IF DATA EXISTS)
# ============================================================================

# Path to preprocessed data
PREPROCESSED_DATA_DIR = "/space/mcdonald-syn01/1/projects/jsawant/llm_recommender_2/data/processed_notebook"

# Check if preprocessed data exists
preprocessed_path = Path(PREPROCESSED_DATA_DIR)
if preprocessed_path.exists() and (preprocessed_path / 'train_data.pkl').exists():
    print(f"Loading preprocessed data from {PREPROCESSED_DATA_DIR}...")
    
    # Load processed data
    with open(preprocessed_path / 'train_data.pkl', 'rb') as f:
        train_data = pickle.load(f)
    with open(preprocessed_path / 'val_data.pkl', 'rb') as f:
        val_data = pickle.load(f)
    with open(preprocessed_path / 'test_data.pkl', 'rb') as f:
        test_data = pickle.load(f)
    
    # Load dataset info
    with open(preprocessed_path / 'dataset_info.pkl', 'rb') as f:
        dataset_info = pickle.load(f)
    
    # Load mappings
    with open(preprocessed_path / 'item_metadata.pkl', 'rb') as f:
        item_metadata = pickle.load(f)
    
    # Extract key info
    num_users = dataset_info['num_users']
    num_items = dataset_info['num_items']
    
    print(f"\nPreprocessed data loaded successfully!")
    print(f"  Users: {num_users}")
    print(f"  Items: {num_items}")
    print(f"  Train sequences: {dataset_info['num_train']}")
    print(f"  Val sequences: {dataset_info['num_val']}")
    print(f"  Test sequences: {dataset_info['num_test']}")
    print(f"  Item metadata: {len(item_metadata)} items")
    
    # Update DATA_OUTPUT_DIR to match loaded data location
    DATA_OUTPUT_DIR = str(preprocessed_path)
    print(f"\nUsing data from: {DATA_OUTPUT_DIR}")
else:
    print(f"Preprocessed data not found at {PREPROCESSED_DATA_DIR}")
    print("Please run the preprocessing cell (Step 1) first, or update PREPROCESSED_DATA_DIR path.")
    print("Skipping data loading - you'll need to run preprocessing.")


Loading preprocessed data from /space/mcdonald-syn01/1/projects/jsawant/llm_recommender_2/data/processed_notebook...

Preprocessed data loaded successfully!
  Users: 100000
  Items: 389162
  Train sequences: 100000
  Val sequences: 100000
  Test sequences: 100000
  Item metadata: 389161 items

Using data from: /space/mcdonald-syn01/1/projects/jsawant/llm_recommender_2/data/processed_notebook


## Step 2: Create Data Loaders

Create PyTorch data loaders for training, validation, and testing.


In [ ]:
# ============================================================================
# CREATE DATA LOADERS
# ============================================================================

data_module = RecDataModule(
    train_data=train_data,
    val_data=val_data,
    test_data=test_data,
    item_metadata=item_metadata,
    num_items=num_items,
    tokenizer=None,  # No tokenizer needed for Stage A
    batch_size=CONFIG['stage_a']['batch_size'],
    num_workers=4,
    negative_samples=CONFIG['stage_a']['collaborative']['negative_samples'],
    max_seq_length=CONFIG['data']['max_sequence_length'],
    max_text_length=None,
    eval_seed=CONFIG['data']['seed']
)

train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()
test_loader = data_module.test_dataloader()

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")


## Step 3: Initialize Model

Create and initialize the Stage A model.


In [12]:
# ============================================================================
# INITIALIZE MODEL
# ============================================================================

model = StageAModel(
    base_llm_name=CONFIG['model']['base_llm'],
    num_users=num_users,
    num_items=num_items,
    embedding_dim=CONFIG['model']['embedding_dim'],
    lambda_c=CONFIG['embeddings']['lambda_c'],
    freeze_llm=CONFIG['model']['freeze_llm_stage_a'],
    use_bpr_loss=CONFIG['stage_a']['collaborative'].get('use_bpr_loss', True),
    random_init_llm=CONFIG['model'].get('random_init_stage_a_llm', False)
)

model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nModel parameters:")
print(f"  Total: {total_params:,}")
print(f"  Trainable: {trainable_params:,}")

# Create optimizer
optimizer = AdamW(
    model.parameters(),
    lr=float(CONFIG['stage_a']['learning_rate']),
    weight_decay=float(CONFIG['stage_a']['weight_decay'])
)

print("\nModel and optimizer initialized!")



Model parameters:
  Total: 187,347,456
  Trainable: 187,347,456

Model and optimizer initialized!


## Step 4: Training

Train the model. This includes training loop, evaluation, and checkpoint saving.


In [13]:
# ============================================================================
# TRAINING LOOP
# ============================================================================

# Training configuration
loss_weights = CONFIG['stage_a']['loss_weights']
max_grad_norm = CONFIG['stage_a']['max_grad_norm']
mixed_precision = CONFIG['training']['mixed_precision']
logging_steps = CONFIG['training']['logging_steps']
eval_steps = CONFIG['training']['eval_steps']
save_steps = CONFIG['training']['save_steps']

# Setup mixed precision
use_amp = mixed_precision in ['fp16', 'bf16']
amp_dtype = torch.float16 if mixed_precision == 'fp16' else (
    torch.bfloat16 if mixed_precision == 'bf16' else torch.float32
)
device_type = 'cuda' if 'cuda' in str(device) else 'cpu'
scaler = GradScaler(device_type) if mixed_precision == 'fp16' and device_type == 'cuda' else None

# Training state
global_step = 0
best_metric = 0.0
epoch_history = []

# Save config
with open(Path(CHECKPOINT_DIR) / 'config.json', 'w') as f:
    json.dump(CONFIG, f, indent=2)

print("Starting training...")
print(f"Output directory: {CHECKPOINT_DIR}")

for epoch in range(CONFIG['stage_a']['epochs']):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch + 1}/{CONFIG['stage_a']['epochs']}")
    print(f"{'='*60}")
    
    # Training
    model.train()
    total_loss = 0.0
    component_sums = {'collab_ce': 0.0, 'bpr': 0.0, 'regularization': 0.0}
    component_counts = {k: 0 for k in component_sums.keys()}
    
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")
    
    for batch_idx, batch in enumerate(progress_bar):
        batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v 
                for k, v in batch.items()}
        
        optimizer.zero_grad()
        
        with autocast(device_type=device_type, enabled=use_amp, dtype=amp_dtype):
            is_autoregressive = batch.get('is_autoregressive', False)
            collab_losses = model.compute_collaborative_loss(
                user_ids=batch['user_ids'],
                item_ids=batch['item_ids'],
                attention_mask=batch['attention_mask'],
                target_items=batch['target_item_ids'],
                is_autoregressive=is_autoregressive,
                negative_items=batch.get('negative_items')
            )
            
            reg_losses = model.compute_regularization_losses()
            
            loss = 0.0
            loss_components = {}
            
            if 'collaborative_ce' in collab_losses:
                loss += loss_weights['collaborative'] * collab_losses['collaborative_ce']
                loss_components['collab_ce'] = collab_losses['collaborative_ce'].item()
            
            if 'bpr' in collab_losses:
                loss += loss_weights.get('cf_bpr', 1.0) * collab_losses['bpr']
                loss_components['bpr'] = collab_losses['bpr'].item()
            
            reg_weight = loss_weights.get('regularization', 0.0)
            if reg_weight > 0:
                loss += reg_weight * reg_losses['collab_regularization']
                loss_components['regularization'] = reg_losses['collab_regularization'].item()
            else:
                loss_components['regularization'] = reg_losses['collab_regularization'].item()
        
        # Backward pass
        if scaler is not None:
            scaler.scale(loss).backward()
            if max_grad_norm > 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            if max_grad_norm > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
        
        total_loss += loss.item()
        
        if 'collab_ce' in loss_components:
            component_sums['collab_ce'] += loss_components['collab_ce']
            component_counts['collab_ce'] += 1
        if 'bpr' in loss_components:
            component_sums['bpr'] += loss_components['bpr']
            component_counts['bpr'] += 1
        if 'regularization' in loss_components:
            component_sums['regularization'] += loss_components['regularization']
            component_counts['regularization'] += 1
        
        global_step += 1
        
        if global_step % logging_steps == 0:
            avg_loss = total_loss / (batch_idx + 1)
            print(f"\n[Step {global_step}] Loss: {loss.item():.4f}, Avg: {avg_loss:.4f}")
            progress_bar.set_postfix({'loss': f'{avg_loss:.4f}', 'step': global_step})
        
        if global_step % save_steps == 0:
            checkpoint_path = Path(CHECKPOINT_DIR) / f"checkpoint-{global_step}"
            checkpoint_path.mkdir(parents=True, exist_ok=True)
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'global_step': global_step,
                'best_metric': best_metric
            }, checkpoint_path / 'pytorch_model.pt')
            torch.save({
                'collab_embeddings': model.collab_embeddings.state_dict(),
            }, checkpoint_path / 'embeddings.pt')
            print(f"Checkpoint saved to {checkpoint_path}")
    
    # Compute average loss components
    avg_components = {}
    for key in component_sums:
        if component_counts[key] > 0:
            avg_components[key] = component_sums[key] / component_counts[key]
    
    train_metrics = {
        'loss': total_loss / len(train_loader),
        **avg_components
    }
    
    print(f"\nEpoch {epoch + 1} training metrics:")
    print(f"  Total loss: {train_metrics.get('loss', 0):.4f}")
    if 'collab_ce' in train_metrics:
        print(f"  Collaborative CE: {train_metrics['collab_ce']:.4f}")
    if 'bpr' in train_metrics:
        print(f"  BPR loss: {train_metrics['bpr']:.4f}")
    
    # Evaluation
    model.eval()
    all_scores = []
    all_target_indices = []
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating"):
            batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v 
                    for k, v in batch.items()}
            
            pos_items = batch['target_item_ids']
            neg_items = batch['negative_items']
            
            with autocast(device_type=device_type, enabled=use_amp, dtype=amp_dtype):
                llm_output = model.forward_collaborative(
                    user_ids=batch['user_ids'],
                    item_ids=batch['item_ids'],
                    attention_mask=batch['attention_mask']
                )
                
                candidates = torch.cat([
                    pos_items.unsqueeze(1),
                    neg_items
                ], dim=1)
                
                candidate_scores = model.collab_scoring_head(
                    llm_output, 
                    candidate_items=candidates,
                    use_fusion=True
                )
                
                batch_target_idx = torch.zeros(
                    candidate_scores.size(0),
                    dtype=torch.long,
                    device=candidate_scores.device
                )
                
                all_scores.append(candidate_scores.cpu())
                all_target_indices.append(batch_target_idx.cpu())
    
    if all_scores:
        all_scores = torch.cat(all_scores, dim=0)
        all_target_indices = torch.cat(all_target_indices, dim=0)
        val_metrics = compute_sampled_metrics(all_scores, all_target_indices, k_list=[1, 5, 10, 20])
        print_metrics(val_metrics, prefix=f"Epoch {epoch + 1} Validation ")
        
        # Update best metric
        if val_metrics['hit@10'] > best_metric:
            best_metric = val_metrics['hit@10']
            best_checkpoint_path = Path(CHECKPOINT_DIR) / 'best_model'
            best_checkpoint_path.mkdir(parents=True, exist_ok=True)
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'global_step': global_step,
                'best_metric': best_metric
            }, best_checkpoint_path / 'pytorch_model.pt')
            torch.save({
                'collab_embeddings': model.collab_embeddings.state_dict(),
            }, best_checkpoint_path / 'embeddings.pt')
            print(f"  ✓ New best model saved! Hit@10: {val_metrics['hit@10']:.4f}")
        
        # Save epoch metrics
        record = {
            "epoch": epoch + 1,
            "global_step": float(global_step),
        }
        for key, value in train_metrics.items():
            record[f"train_{key}"] = float(value)
        for key, value in val_metrics.items():
            record[f"val_{key}"] = float(value)
        epoch_history.append(record)
        
        with open(Path(CHECKPOINT_DIR) / 'metrics.json', 'w') as f:
            json.dump(epoch_history, f, indent=2)

print("\n" + "=" * 60)
print("Training complete!")
print(f"Best Hit@10: {best_metric:.4f}")
print("=" * 60)


Starting training...
Output directory: ./checkpoints/notebook_training

Epoch 1/10


Epoch 1:  32%|███▏      | 500/1563 [01:54<03:27,  5.13it/s, loss=20.3124, step=500]


[Step 500] Loss: 19.6449, Avg: 20.3124


Epoch 1:  64%|██████▍   | 1001/1563 [03:47<02:07,  4.39it/s, loss=19.9729, step=1000]


[Step 1000] Loss: 19.7887, Avg: 19.9729


...


[Step 15500] Loss: 7.8128, Avg: 7.7294


Epoch 10: 100%|██████████| 1563/1563 [07:21<00:00,  3.54it/s, loss=7.7294, step=15500]



Epoch 10 training metrics:
  Total loss: 7.7677
  Collaborative CE: 4.1322
  BPR loss: 18.1777


Evaluating: 100%|██████████| 1563/1563 [00:35<00:00, 43.55it/s]



Epoch 10 Validation Metrics:
--------------------------------------------------
  hit@1: 0.2834
  hit@10: 0.5534
  hit@20: 0.6494
  hit@5: 0.4697
  ndcg@1: 0.2834
  ndcg@10: 0.4089
  ndcg@20: 0.4331
  ndcg@5: 0.3818
--------------------------------------------------

Training complete!
Best Hit@10: 0.5986


## Step 5: Evaluation/Prediction

Evaluate the trained model on test data. You can also load a previously saved checkpoint.


In [ ]:
# ============================================================================
# EVALUATION ON TEST DATA
# ============================================================================

# Option 1: Use the model from training (already loaded)
# Option 2: Load from checkpoint (uncomment to use)
# checkpoint_path = Path(CHECKPOINT_DIR) / 'best_model'
# checkpoint = torch.load(checkpoint_path / 'pytorch_model.pt', map_location='cpu', weights_only=False)
# model.load_state_dict(checkpoint['model_state_dict'], strict=False)
# print(f"Loaded checkpoint from {checkpoint_path}")

model.eval()
all_scores = []
all_target_indices = []

use_amp = mixed_precision in ['fp16', 'bf16']
amp_dtype = torch.float16 if mixed_precision == 'fp16' else (
    torch.bfloat16 if mixed_precision == 'bf16' else torch.float32
)
device_type = 'cuda' if 'cuda' in str(device) else 'cpu'

print("Evaluating on test data...")
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Test evaluation"):
        batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v 
                for k, v in batch.items()}
        
        pos_items = batch['target_item_ids']
        neg_items = batch['negative_items']
        
        with autocast(device_type=device_type, enabled=use_amp, dtype=amp_dtype):
            llm_output = model.forward_collaborative(
                user_ids=batch['user_ids'],
                item_ids=batch['item_ids'],
                attention_mask=batch['attention_mask']
            )
            
            candidates = torch.cat([
                pos_items.unsqueeze(1),
                neg_items
            ], dim=1)
            
            candidate_scores = model.collab_scoring_head(
                llm_output, 
                candidate_items=candidates,
                use_fusion=True
            )
            
            batch_target_idx = torch.zeros(
                candidate_scores.size(0),
                dtype=torch.long,
                device=candidate_scores.device
            )
            
            all_scores.append(candidate_scores.cpu())
            all_target_indices.append(batch_target_idx.cpu())

if all_scores:
    all_scores = torch.cat(all_scores, dim=0)
    all_target_indices = torch.cat(all_target_indices, dim=0)
    test_metrics = compute_sampled_metrics(all_scores, all_target_indices, k_list=[1, 5, 10, 20])
    
    print("\n" + "=" * 60)
    print("Test Results:")
    print("=" * 60)
    print_metrics(test_metrics, prefix="Test ")
    
    # Save test metrics
    with open(Path(CHECKPOINT_DIR) / 'test_metrics.json', 'w') as f:
        json.dump(test_metrics, f, indent=2)
    print(f"\nTest metrics saved to {Path(CHECKPOINT_DIR) / 'test_metrics.json'}")


## Optional: Load and Predict from Saved Checkpoint

If you want to load a previously saved model and make predictions, use this cell.


In [17]:
# ============================================================================
# LOAD CHECKPOINT AND PREDICT
# ============================================================================

# Uncomment and modify the path to load a specific checkpoint
CHECKPOINT_TO_LOAD = "/space/mcdonald-syn01/1/projects/jsawant/llm_recommender/checkpoints/stage_a_100_v10/best_model"

# Load data
with open(Path(DATA_OUTPUT_DIR) / 'dataset_info.pkl', 'rb') as f:
    dataset_info = pickle.load(f)
with open(Path(DATA_OUTPUT_DIR) / 'test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)
with open(Path(DATA_OUTPUT_DIR) / 'item_metadata.pkl', 'rb') as f:
    item_metadata = pickle.load(f)

num_users = dataset_info['num_users']
num_items = dataset_info['num_items']

# Load config from checkpoint or use CONFIG
try:
    with open(Path(CHECKPOINT_TO_LOAD).parent / 'config.json', 'r') as f:
        loaded_config = json.load(f)
    CONFIG = loaded_config
except:
    print("Using current CONFIG")

# Create model
model = StageAModel(
    base_llm_name=CONFIG['model']['base_llm'],
    num_users=num_users,
    num_items=num_items,
    embedding_dim=CONFIG['model']['embedding_dim'],
    lambda_c=CONFIG['embeddings']['lambda_c'],
    freeze_llm=CONFIG['model']['freeze_llm_stage_a'],
    use_bpr_loss=CONFIG['stage_a']['collaborative'].get('use_bpr_loss', True),
    random_init_llm=CONFIG['model'].get('random_init_stage_a_llm', False)
)

# Load checkpoint
checkpoint = torch.load(
    Path(CHECKPOINT_TO_LOAD) / 'pytorch_model.pt',
    map_location='cpu',
    weights_only=False
)
model.load_state_dict(checkpoint['model_state_dict'], strict=False)
model = model.to(device)
model.eval()

print(f"Model loaded from {CHECKPOINT_TO_LOAD}")

# Create test data loader
data_module = RecDataModule(
    train_data={},
    val_data={},
    test_data=test_data,
    item_metadata=item_metadata,
    num_items=num_items,
    tokenizer=None,
    batch_size=32,
    num_workers=4,
    negative_samples=100,
    max_seq_length=CONFIG['data']['max_sequence_length'],
    max_text_length=None,
    eval_seed=CONFIG['data']['seed']
)
test_loader = data_module.test_dataloader()

# Evaluate
all_scores = []
all_target_indices = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v 
                for k, v in batch.items()}
        
        pos_items = batch['target_item_ids']
        neg_items = batch['negative_items']
        
        with autocast(device_type=device_type, enabled=use_amp, dtype=amp_dtype):
            llm_output = model.forward_collaborative(
                user_ids=batch['user_ids'],
                item_ids=batch['item_ids'],
                attention_mask=batch['attention_mask']
            )
            
            candidates = torch.cat([
                pos_items.unsqueeze(1),
                neg_items
            ], dim=1)
            
            candidate_scores = model.collab_scoring_head(
                llm_output, 
                candidate_items=candidates,
                use_fusion=True
            )
            
            batch_target_idx = torch.zeros(
                candidate_scores.size(0),
                dtype=torch.long,
                device=candidate_scores.device
            )
            
            all_scores.append(candidate_scores.cpu())
            all_target_indices.append(batch_target_idx.cpu())

if all_scores:
    all_scores = torch.cat(all_scores, dim=0)
    all_target_indices = torch.cat(all_target_indices, dim=0)
    test_metrics = compute_sampled_metrics(all_scores, all_target_indices, k_list=[1, 5, 10, 20])
    print_metrics(test_metrics, prefix="Test ")

print("This cell is for loading checkpoints. Uncomment and modify the code above to use it.")


Model loaded from /space/mcdonald-syn01/1/projects/jsawant/llm_recommender/checkpoints/stage_a_100_v10/best_model


Precomputing negative samples: 0it [00:00, ?it/s]
Evaluating: 100%|██████████| 3125/3125 [00:36<00:00, 84.81it/s]


Test Metrics:
--------------------------------------------------
  hit@1: 0.2846
  hit@10: 0.5577
  hit@20: 0.6436
  hit@5: 0.4781
  ndcg@1: 0.2846
  ndcg@10: 0.4127
  ndcg@20: 0.4344
  ndcg@5: 0.3870
--------------------------------------------------
This cell is for loading checkpoints. Uncomment and modify the code above to use it.
